全波形反演为匹配整个记录数据集(包括折射波)的模型提供了反演的潜力。

我们继续使用Marmousi模型，但由于运行多次FWI需要更大的计算成本，这里，我们将只研究其中的一部分：

In [ ]:
import torch
from torchaudio.functional import biquad
from scipy.ndimage import gaussian_filter
from scipy.signal import butter
import matplotlib.pyplot as plt
import deepwave
from deepwave import scalar

device = torch.device('cuda' if torch.cuda.is_available()
                      else 'cpu')
ny = 2301
nx = 751
dx = 4.0
v_true = torch.from_file('marmousi_vp.bin',
                         size=ny*nx).reshape(ny, nx)

# Select portion of model for inversion
ny = 600
nx = 250
v_true = v_true[:ny, :nx]

我们对真实模型进行平滑来创建初始速度模型，并尝试通过反演来改进它。使用上一个例子中的合成数据为观测数据：

In [ ]:
v_init = (torch.tensor(1/gaussian_filter(1/v_true.numpy(), 40))
          .to(device))
v = v_init.clone()
v.requires_grad_()

n_shots = 115

n_sources_per_shot = 1
d_source = 20  # 20 * 4m = 80m
first_source = 10  # 10 * 4m = 40m
source_depth = 2  # 2 * 4m = 8m

n_receivers_per_shot = 384
d_receiver = 6  # 6 * 4m = 24m
first_receiver = 0  # 0 * 4m = 0m
receiver_depth = 2  # 2 * 4m = 8m

freq = 25
nt = 750
dt = 0.004
peak_time = 1.5 / freq

observed_data = (
    torch.from_file('marmousi_data.bin',
                    size=n_shots*n_receivers_per_shot*nt)
    .reshape(n_shots, n_receivers_per_shot, nt)
)

由于我们的模型现在更小了，我们也只需要提取覆盖模型这一部分的观测数据：

In [ ]:
n_shots = 20
n_receivers_per_shot = 100
nt = 300
observed_data = (
    observed_data[:n_shots, :n_receivers_per_shot, :nt].to(device)
)

我们像之前一样设置震源和接收器：

In [ ]:
# source_locations
source_locations = torch.zeros(n_shots, n_sources_per_shot, 2,
                               dtype=torch.long, device=device)
source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = (torch.arange(n_shots) * d_source +
                             first_source)

# receiver_locations
receiver_locations = torch.zeros(n_shots, n_receivers_per_shot, 2,
                                 dtype=torch.long, device=device)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = (
    (torch.arange(n_receivers_per_shot) * d_receiver +
     first_receiver)
    .repeat(n_shots, 1)
)

# source_amplitudes
source_amplitudes = (
    (deepwave.wavelets.ricker(freq, nt, dt, peak_time))
    .repeat(n_shots, n_sources_per_shot, 1).to(device)
)

现在我们准备运行优化器来执行波速模型的迭代反演。这将使用与PyTorch中用于训练典型神经网络非常相似的代码来实现。唯一值得注意的区别是，我们使用更大的学习率(1e9)将梯度值提升到一个范围，这将有助于我们在每次迭代中取得良好的进展，并且我们对梯度应用剪辑(到其大小的第98个百分位数)，以避免在少数点(例如源周围)进行非常大的更改：

In [ ]:
# Setup optimiser to perform inversion
optimiser = torch.optim.SGD([v], lr=1e9, momentum=0.9)
loss_fn = torch.nn.MSELoss()

# Run optimisation/inversion
n_epochs = 250
v_true = v_true.to(device)

for epoch in range(n_epochs):
    optimiser.zero_grad()
    out = scalar(
        v, dx, dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        pml_freq=freq,
    )
    loss = loss_fn(out[-1], observed_data)
    loss.backward()
    torch.nn.utils.clip_grad_value_(
        v,
        torch.quantile(v.grad.detach().abs(), 0.98)
    )
    optimiser.step()

结果提高了我们对波速模型的估计精度。请注意，源没有覆盖右侧表面的一部分，这就是为什么那里的结果更差。

然而，看起来低波数信息(层速度)在模型的较低部分丢失了。这可能是由于通常影响地震反演的周期跳变问题。

In [ ]:
# Plot
vmin = v_true.min()
vmax = v_true.max()
_, ax = plt.subplots(3, figsize=(10.5, 10.5), sharex=True,
                     sharey=True)
ax[0].imshow(v_init.cpu().T, aspect='auto', cmap='jet',
             vmin=vmin, vmax=vmax)
ax[0].set_title("Initial")
ax[1].imshow(v.detach().cpu().T, aspect='auto', cmap='jet',
             vmin=vmin, vmax=vmax)
ax[1].set_title("Out")
ax[2].imshow(v_true.cpu().T, aspect='auto', cmap='jet',
             vmin=vmin, vmax=vmax)
ax[2].set_title("True")
plt.tight_layout()
plt.savefig('example_simple_fwi.jpg')

这样我们就遇到了两个问题。首先是在逆温过程中，特定单元的速度值变得太大或太小的风险，并引起稳定性问题，这是由源和接收器附近的梯度值很大引起的，因此优化器在那里采取了很大的步骤。我们试图通过使用渐变剪辑来解决这个问题。第二个问题是周期跳变，即模型的到达点与目标对应点之间的偏移超过半个波长，这可能导致反演陷入局部最小值。让我们尝试通过对代码进行一些改进来克服这些问题。

解决速度极值问题的一种方法是将速度限制在期望的范围内。因为PyTorch允许我们将运算符链在一起，并且会自动通过它们反向传播来计算梯度，所以我们可以使用函数来生成速度模型。这为我们提供了一种方便而稳健的方法来约束模型中的速度范围。我们可以将我们的速度模型定义为一个包含与我们的模型大小相同的张量的物体。当我们调用这个对象的forward方法时，它返回对这个存储张量应用sigmoid操作的输出，结果是每个单元格的值在0到1之间，然后将其缩放到我们想要的范围。我们可以使用logit算子将它的初始输出设置为我们选择的初始速度模型，它是sigmoid的逆:

In [ ]:
## Second attempt: constrained velocity and frequency filtering


# Define a function to taper the ends of traces
def taper(x):
    return deepwave.common.cosine_taper_end(x, 100)

In [ ]:
# Generate a velocity model constrained to be within a desired range
class Model(torch.nn.Module):
    def __init__(self, initial, min_vel, max_vel):
        super().__init__()
        self.min_vel = min_vel
        self.max_vel = max_vel
        self.model = torch.nn.Parameter(
            torch.logit((initial - min_vel) /
                        (max_vel - min_vel))
        )

    def forward(self):
        return (torch.sigmoid(self.model) *
                (self.max_vel - self.min_vel) +
                self.min_vel)


observed_data = taper(observed_data)
model = Model(v_init, 1000, 2500).to(device)


In [ ]:
# Run optimisation/inversion
n_epochs = 2

for cutoff_freq in [10, 15, 20, 25, 30]:
    sos = butter(6, cutoff_freq, fs=1/dt, output='sos')
    sos = [torch.tensor(sosi).to(observed_data.dtype).to(device)
           for sosi in sos]

    def filt(x):
        return biquad(biquad(biquad(x, *sos[0]), *sos[1]), *sos[2])
    observed_data_filt = filt(observed_data)
    optimiser = torch.optim.LBFGS(model.parameters(),
                                  line_search_fn='strong_wolfe')
    for epoch in range(n_epochs):
        def closure():
            optimiser.zero_grad()
            v = model()
            out = scalar(
                v, dx, dt,
                source_amplitudes=source_amplitudes,
                source_locations=source_locations,
                receiver_locations=receiver_locations,
                max_vel=2500,
                pml_freq=freq,
                time_pad_frac=0.2,
            )
            out_filt = filt(taper(out[-1]))
            loss = 1e6*loss_fn(out_filt, observed_data_filt)
            loss.backward()
            return loss

        optimiser.step(closure)

In [ ]:
# Plot
v = model()
vmin = v_true.min()
vmax = v_true.max()
_, ax = plt.subplots(3, figsize=(10.5, 10.5), sharex=True,
                     sharey=True)
ax[0].imshow(v_init.cpu().T, aspect='auto', cmap='gray',
             vmin=vmin, vmax=vmax)
ax[0].set_title("Initial")
ax[1].imshow(v.detach().cpu().T, aspect='auto', cmap='gray',
             vmin=vmin, vmax=vmax)
ax[1].set_title("Out")
ax[2].imshow(v_true.cpu().T, aspect='auto', cmap='gray',
             vmin=vmin, vmax=vmax)
ax[2].set_title("True")
plt.tight_layout()
plt.savefig('example_increasing_freq_fwi.jpg')